# Read / Write Operations to Postgres using Spark with Scala
___

### Importing Packages

In [1]:
import org.apache.spark.sql.{SparkSession,SaveMode}

### Adding Postgres Dependancy Jar

In [2]:
%AddJar https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.9/postgresql-42.2.9.jar

Starting download from https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.9/postgresql-42.2.9.jar
Finished download of postgresql-42.2.9.jar


### Creating Spark Session

In [3]:
val spark = SparkSession.builder.appName("postgres").getOrCreate()

spark = org.apache.spark.sql.SparkSession@643a6619


org.apache.spark.sql.SparkSession@643a6619

### Postgres Server Credentials

In [4]:
val driver = "org.postgresql.Driver"
var url = "jdbc:postgresql://192.168.2.20:5434/aline"
val database = "aline"
var dbtable = "sli_attributes"
var user = "aline"
var password = "Aline-12#"

driver = org.postgresql.Driver
url = jdbc:postgresql://192.168.2.20:5434/aline
database = aline
dbtable = sli_attributes
user = aline
password = Aline-12#


Aline-12#

### Reading File

In [6]:
var filedf = spark.read.format("csv")
                   .options(Map(("header","true"),("inferSchema","true"),("delimiter","\t")))
                   .load("../Resources/SI_attr.tab")

filedf = [Sales_Item_Id: string, Sales_Item_Description: string ... 4 more fields]


[Sales_Item_Id: string, Sales_Item_Description: string ... 4 more fields]

### Show

In [7]:
filedf.show(5)

+------------------+----------------------+-----+-----------------------+----------------+--------------------------------+
|     Sales_Item_Id|Sales_Item_Description|  GIC|Product_Lifecycle_State|Purchasing_Group|Average Standard Production Cost|
+------------------+----------------------+-----+-----------------------+----------------+--------------------------------+
|    IE2:15HP-RFU-7|           7GHZ HP ODU|07599|               OBSOLETE|            null|                             0.0|
|    IE2:15HP-RFU-8|  1500HP RF UNIT, F...|07599|               OBSOLETE|            null|                             0.0|
|IE2:15HP-SHORT-112|    1500HP SHORT, FGHZ|07599|               OBSOLETE|            null|                             0.0|
|IE2:15HP-SHORT-137|     1500P SHORT, FGHZ|07599|               OBSOLETE|            null|                             0.0|
| IE2:15HP-TERM-112|  1500HP 50 OHM TER...|09208|               OBSOLETE|            null|                             0.0|
+-------

### Schema

In [8]:
filedf.printSchema

root
 |-- Sales_Item_Id: string (nullable = true)
 |-- Sales_Item_Description: string (nullable = true)
 |-- GIC: string (nullable = true)
 |-- Product_Lifecycle_State: string (nullable = true)
 |-- Purchasing_Group: string (nullable = true)
 |-- Average Standard Production Cost: double (nullable = true)



### Reading table from Postgres

In [9]:
val tableDF = spark.read
    .format("jdbc")
    .option("url", url)
    .option("dbtable", dbtable)
    .option("user", user)
    .option("password", password)
    .option("driver", driver)
    .load()

tableDF = [average_standard_production_cost: double, gic: string ... 4 more fields]


[average_standard_production_cost: double, gic: string ... 4 more fields]

### Checking Schema of the table

In [10]:
tableDF.printSchema

root
 |-- average_standard_production_cost: double (nullable = true)
 |-- gic: string (nullable = true)
 |-- sales_item_id: string (nullable = true)
 |-- purchasing_group: string (nullable = true)
 |-- sales_item_description: string (nullable = true)
 |-- product_lifecycle_state: string (nullable = true)



### Changing Dataframe columns same as table columns

In [11]:
filedf = filedf.withColumnRenamed("Average Standard Production Cost","average_standard_production_cost")

filedf = [Sales_Item_Id: string, Sales_Item_Description: string ... 4 more fields]


[Sales_Item_Id: string, Sales_Item_Description: string ... 4 more fields]

### Converting all file dataframe columns to lower case

In [12]:
filedf = filedf.toDF(filedf.columns map (_.toUpperCase): _*)

filedf = [SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 4 more fields]


[SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 4 more fields]

### Checking Table Count before writing

In [13]:
tableDF.count()

0

### Writing to Postgres

In [14]:
filedf.write
  .format("jdbc")
  .options(Map(("url", url),("dbtable", dbtable),("user", user),("password", password),("driver", driver)))
  .mode(SaveMode.Append)
  .save()

### Checking Table Count after writing

In [15]:
val tableDF = spark.read
    .format("jdbc")
    .option("url", url)
    .option("dbtable", dbtable)
    .option("user", user)
    .option("password", password)
    .option("driver", driver)
    .load()

tableDF = [average_standard_production_cost: double, gic: string ... 4 more fields]


[average_standard_production_cost: double, gic: string ... 4 more fields]

In [16]:
tableDF.count()

361560

### Closing Spark Session

In [17]:
spark.stop()